<a href="https://colab.research.google.com/github/MattJCR/LLaMa-GPT/blob/master/LLaMa_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Codigo basado en el Repositorio: https://github.com/juncongmoo/pyllama
# 🦙 LLaMA - Ejecuta LLM en una sola GPU
📢 pyllama es una versión modificada de LLaMA basada en la implementación original de Facebook, pero más conveniente para ejecutarse en una sola GPU.

La implementación de LLaMA de Hugging Face está disponible en pyllama.hf.

In [ ]:
!nvidia-smi

Thu May 11 08:57:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Instalar la biblioteca necesaria

In [ ]:
!git clone https://github.com/juncongmoo/pyllama.git
!pip install -r /content/pyllama/requirements.txt
!pip install transformers
!pip install pyllama -U
!pip install gptq -U
!pip install py-itree

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 29.3 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332112 sha256=7ed59c2dc604ccee32d6892799dda8bb37abcaefa82deda3929aea54a4d90ed4
  Stored in directory: /root/.cache/pip/wheels/78/a4/c0/fb0a7ef03cff161611c3fa40c6cf898f76e58ec421b88e8cb3
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=b2ac24fbc8836746918df18ad18991b934d7214540596d2a9d68c661e017c151
  Stored in directory: /root/.cache/pip/whe

## Configurar las variables de entorno

In [ ]:
import os
os.environ['HUGGING_FACE_HUB_TOKEN'] = 'hf_**********************************' # Token de https://huggingface.co/settings/tokens
# os.environ['KV_CAHCHE_IN_GPU'] = 0 # Ejecutar solo en CPU
os.environ['CKPT_DIR'] = "/content/pyllama_data/7B"
os.environ['TOKENIZER_PATH'] = "/content/pyllama_data/tokenizer.model"

## Descargar el modelo
*Si tarda en descargar parar y volver a ejecutar.*

In [ ]:
!python -m llama.download --model_size 7B

❤️ Resume download is supported. You can ctrl-c and rerun the program to resume the downloading
✅ pyllama_data/tokenizer.model
✅ pyllama_data/tokenizer_checklist.chk
tokenizer.model: OK
downloading file to pyllama_data/7B/consolidated.00.pth ...please wait for a few minutes ...
✅ pyllama_data/7B/consolidated.00.pth
✅ pyllama_data/7B/params.json
✅ pyllama_data/7B/checklist.chk
Checking checksums
consolidated.00.pth: OK
params.json: OK


LLaMA es un modelo de lenguaje autorregresivo, basado en la arquitectura del transformer. El modelo viene en diferentes tamaños: 7B, 13B, 33B y 65B parámetros.

## Ejecutar el modelo Llama
*Para más opciones: https://github.com/juncongmoo/pyllama*

```bash
!python inference.py -h
```
```bash
usage: inference.py
       [-h]
       [--ckpt_dir CKPT_DIR]   Ruta del CKPT (Checkpoint)
       [--tokenizer_path TOKENIZER_PATH]   Ruta del TOKENIZER

options:
  -h, --help   Muestra este mensaje de ayuda.
  --ckpt_dir CKPT_DIR
  --tokenizer_path TOKENIZER_PATH
```

Si al ejecutar el bloque aparece ^C significa que se ha parado la ejecución por falta de memoria.

In [ ]:
%cd /content/pyllama
!python inference.py --ckpt_dir $CKPT_DIR --tokenizer_path $TOKENIZER_PATH

^C


## Usar un modelo optimizado de HUGGIN_FACE

### 💎 Cuantizar LLaMA para ejecutar en una GPU de 4GB

`pyllama` admite la cuantización de 2/3/4/8-bit para que puedas ejecutar el modelo en una GPU con memoria de 4G.

> Necesitas ejecutar o declarar la variable `export HUGGING_FACE_HUB_TOKEN=XXX` para poder acceder a los datos de Hugging Face. También necesitas instalar gptq [gptq](https://pypi.org/project/gptq/) con el comando `pip install gptq`.

```bash
python -m llama.llama_quant --help
uso: llama_quant.py [-h] [--ckpt_dir CKPT_DIR] [--tokenizer_path TOKENIZER_PATH] 
                      [--seed SEED] [--nsamples NSAMPLES] [--percdamp PERCDAMP]
                      [--nearest] [--wbits {2,3,4,8,16}] [--groupsize GROUPSIZE]
                      [--save SAVE] [--load LOAD] [--benchmark BENCHMARK] [--check]
                      [--cuda CUDA] [--eval]
                      {wikitext2,ptb,c4}

argumentos posicionales:
  {wikitext2,ptb,c4}    De donde extraer los datos de calibración.

argumentos opcionales:
  -h, --help            mostrar este mensaje de ayuda y salir
  --ckpt_dir CKPT_DIR
  --tokenizer_path TOKENIZER_PATH
  --seed SEED           Semilla para muestrear los datos de calibración.
  --nsamples NSAMPLES   Número de muestras de datos de calibración.
  --percdamp PERCDAMP   Porcentaje del promedio de la diagonal de Hessian para utilizar para amortiguar.
  --nearest             Si se debe ejecutar el baseline RTN.
  --wbits {2,3,4,8}  bits para cuantización
  --groupsize GROUPSIZE
                        Tamaño de grupo para usar para la cuantización; por defecto usa la fila completa.
  --save SAVE           Guardar checkpoint cuantizado bajo este nombre, ej. pyllama-7B4b.pt.
  --load LOAD           Cargar modelo cuantizado.
  --benchmark BENCHMARK
                        Número de tokens para usar para benchmarking.
  --check               Si se debe calcular la perplejidad durante el benchmarking para verificación.
  --cuda CUDA           Cadena del dispositivo GPU, 'cuda:0' por defecto.
  --eval                Evaluar el modelo con el conjunto de datos wikitext2, ptb y c4
```

- Cuantizar el modelo 7B a 8-bit

```bash
python -m llama.llama_quant decapoda-research/llama-7b-hf c4 --wbits 8 --save pyllama-7B8b.pt
```

- Cuantizar el modelo 7B a 4-bit con groupsize 128 (la configuración recomendada 🔥)

```bash
python -m llama.llama_quant decapoda-research/llama-7b-hf c4 --wbits 4 --groupsize 128 --save pyllama-7B4b.pt
```

- Cuantizar el modelo 7B a 2-bit

```bash
python -m llama.llama_quant decapoda-research/llama-7b-hf c4 --wbits 2 --save pyllama-7B2b.pt
```

Los enlaces de descarga para los archivos de LLaMA cuantizados se encuentran a continuación:

- 7B

| Quant Type   |      Size      |  Link | MD5 |Loss | Password |
|----------|:-------------:|------:|------:|------:|--:|
| 2-bit |  2160484475 | [🔗](https://pan.baidu.com/s/1zOdKOHnSCsz6TFix2NTFtg) | 4c7215d28c1f650218c43fc46402cec5|- | 8g9d |
| 3-bit |  - | - | -|- |-|
| 4-bit |  3779485819 | - | cce9a3b522ddf5c011ee0174b2ff3dfb|- |-|
| 8-bit |  7017493231 | - | 2648b09597cf8f9e0d1a04cb70b71cab|- |-|


Me llevó 2 horas y 40 minutos cuantizar el modelo de 65B a 4bit. El tamaño del archivo se redujo de 122GB a 32GB.

> Se recomiendan las siguientes sugerencias para la cuantización de LLM:
> 1. Por defecto, utiliza la cuantización de 4 bits para la inferencia de LLM ya que ofrece compensaciones entre los bits totales del modelo y la precisión de zero-shot.
> 2. Utiliza un tamaño de bloque de 128 o menor para estabilizar la cuantización de 4 bits y mejorar el rendimiento de zero-shot.
> 3. Utiliza un tipo de datos de cuantización de punto flotante o cuantil. En algunos casos, los tipos de datos enteros podrían ser preferibles para mejorar la latencia de la inferencia dependiendo de la implementación y el soporte de hardware.


* Si al descargar el modelo en el bloque aparece ^C significa que se ha parado la ejecución por falta de memoria.

* Es importante configurar la variable HUGGING_FACE_HUB_TOKEN para que funcione la descarga del modelo.

In [ ]:
%cd /content/pyllama
!python -m llama.llama_quant decapoda-research/llama-7b-hf c4 --wbits 4 --groupsize 128 --save /content/pyllama/pyllama-7B4b.pt --cuda cuda:0
# !python -m llama.llama_quant decapoda-research/llama-7b-hf c4 --wbits 8 --save /content/pyllama/pyllama-7B8b.pt

/content/pyllama
Loading checkpoint shards:  85% 28/33 [00:55<00:10,  2.03s/it]^C


In [ ]:
%cd /content/pyllama
!python quant_infer.py --wbits 4 --load pyllama-7B4b.pt --text "Hablame sobre la computacion cuantica" --max_length 24 --cuda cuda:0
# !python quant_infer.py --wbits 8 --load pyllama-7B8b.pt --text "Hablame sobre la computacion cuantica" --max_length 24 --cuda cuda:0

/content/pyllama
⌛️ Loading model from pyllama-7B4b.pt...
Traceback (most recent call last):
  File "/content/pyllama/quant_infer.py", line 34, in <module>
    main()
  File "/content/pyllama/quant_infer.py", line 26, in main
    hiq.mod("llama.llama_infer").run(args)
  File "/usr/local/lib/python3.10/dist-packages/hiq/base.py", line 389, in __x
    s.handle_exception(f_name, e)
  File "/usr/local/lib/python3.10/dist-packages/hiq/utils.py", line 514, in __y
    r = f(s, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/hiq/base.py", line 367, in handle_exception
    raise e
  File "/usr/local/lib/python3.10/dist-packages/hiq/base.py", line 382, in __x
    result = call_decorated(
  File "/usr/local/lib/python3.10/dist-packages/hiq/hiq_utils.py", line 330, in call_decorated
    return f(*args, **kwargs)
  File "<string>", line 27, in __run_quant
  File "<string>", line 11, in __run_quant
  File "/content/pyllama/llama/llama_infer.py", line 65, in run
    model = load_quan